In [33]:
file_name = 'WPV_WP_001_202405091735.csv'
# 'Floater_QS_003_202303270630.csv'
# 'SBB_BS_003_202303270530.csv'
# 'Glider_HY-BX_001_Pro002_taskframe.csv'
# '39-319485-001.txt'
# 'FOP_FB_001_Pro003_202303270530.csv'
# 'AWG_HYY_038_202404200530.csv'
# 'WPV_WP_001_202405091735.csv'

In [34]:
if file_name.startswith('Glider') and file_name.endswith('csv'):
    print('收到水下滑翔机Glider任务帧数据')
elif file_name.startswith('Floater') and file_name.endswith('csv'):
    print('收到海面小浮子Floater数据')
elif file_name.startswith('FOP') and file_name.endswith('csv'):
    print('收到易布放固定观测平台FOP数据')
elif file_name.startswith('SBB') and file_name.endswith('csv'):
    print('收到小型海床基SBB数据')
elif file_name.startswith('WPV') and file_name.endswith('csv'):
    print('收到大载荷风驱动航行器WPV数据')
elif file_name.startswith('AWG') and file_name.endswith('csv'):
    print('收到机动运载波浪滑翔器AWG数据')
elif file_name.endswith('txt'):
    print('收到剖面漂流浮标Argo数据')

收到大载荷风驱动航行器WPV数据


In [2]:
import socket
import os
import sys
import time

def send_file(s, filepath):
    file_name = os.path.basename(filepath)
    file_size = os.path.getsize(filepath)
    
    # 发送文件名长度
    file_name_encoded = file_name.encode('utf-8')
    s.send(len(file_name_encoded).to_bytes(2, byteorder='big'))
    
    # 发送文件名
    try:
        print('正在发送文件名...')
        s.send(file_name_encoded)
        print('文件名发送成功')
    except Exception as e:
        print("发送文件名时出错:", e)
        return

    # 发送文件大小
    s.send(file_size.to_bytes(8, byteorder='big'))
    
    # 发送文件数据
    with open(filepath, 'rb') as fp:
        print('开始向服务器发送数据...')
        while True:
            data = fp.read(1024)
            if not data:
                print('{0} 数据传输完成...'.format(file_name))
                break
            s.send(data)
        # 添加延时等待，等待一段时间再关闭连接
        # 否则还没发送完就关闭连接了
        time.sleep(1) # 所以这个值可以变大，以免报错

def socket_client():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(('127.0.0.1', 8001))  # 修改为服务端的IP地址和端口号
    except socket.error as msg:
        print(msg)
        sys.exit(1)
    
    while True:
        filepath = input("请输入文件路径：\n")
        if filepath == 'exit':
            break
        if os.path.isfile(filepath):
            send_file(s, filepath)

    # 等待一段时间以确保服务端发送完数据后客户端不会立即关闭连接
    time.sleep(1) # 根据实际情况调整等待时间

    s.close()

if __name__ == '__main__':
    socket_client()


正在发送文件名...
文件名发送成功
开始向服务器发送数据...
time.csv   数据传输完成...
正在发送文件名...
发送文件名时出错: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。


ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。